In [13]:
import numpy as np
import pandas as pd

def run_cascade_single_population(adj_matrix, thr, seed_node_index):
    infected_nodes = np.zeros((adj_matrix.shape[0]))
    input_to_node = np.sum(adj_matrix, axis=0)
    activation_strengths = np.zeros((adj_matrix.shape[0]))  # Store activation strengths
    
    infected_nodes[seed_node_index] = 1
    activation_strengths[seed_node_index] = 1  # Initialize activation strength for seed node
    
    list_of_infected_nodes_per_iter = []
    list_of_activation_strengths_per_iter = []  # Store activation strengths per iteration
    
    list_of_infected_nodes_per_iter.append(np.where(infected_nodes == 1)[0].tolist())
    list_of_activation_strengths_per_iter.append(activation_strengths.tolist())  # Store initial activation strengths
    
    counter = 0
    
    while int(np.sum(infected_nodes)) < adj_matrix.shape[0]:
        if counter > 30:
            break
        
        indices_of_infected_nodes = np.where(infected_nodes == 1)[0]
        mask_array = np.zeros((adj_matrix.shape))
        mask_array[indices_of_infected_nodes, :] = 1
        
        infected_connections = adj_matrix.copy()
        infected_connections = infected_connections * mask_array
        infected_inputs = np.sum(infected_connections, axis=0)
        infected_nodes_indices = np.where(infected_inputs / input_to_node > thr)[0]
        
        # Update activation strengths for newly infected nodes
        activation_strengths[infected_nodes_indices] += 1
        
        list_of_infected_nodes_per_iter.append(infected_nodes_indices.tolist())
        list_of_activation_strengths_per_iter.append(activation_strengths.tolist())  # Store activation strengths
        
        infected_nodes[infected_nodes_indices] = 1
        counter += 1
        
    return list_of_infected_nodes_per_iter, list_of_activation_strengths_per_iter  # Return activation strengths

        
def find_thr(adj_matrix, starting_thr):
    visited_thresholds_per_node = [[] for _ in range(adj_matrix.shape[0])]  # Initialize empty lists

    for seed_node_index in range(adj_matrix.shape[0]):
        visited_thresholds_per_node[seed_node_index] = []
        thr = starting_thr
        
        for dummy_thr in range(1000):  # Limit the number of iterations
            list_of_infected_nodes_per_iter = run_cascade_single_population(adj_matrix, thr, seed_node_index)
            if len(list_of_infected_nodes_per_iter[-1]) == adj_matrix.shape[0]:
                thr *= 2
                visited_thresholds_per_node[seed_node_index].append(thr)
            elif dummy_thr == 0 and len(list_of_infected_nodes_per_iter[-1]) != adj_matrix.shape[0]:
                thr /= 100.
            else:
                break
        
    max_thresholds_per_node = np.asarray([visited_thresholds_per_node[ii][-1] if visited_thresholds_per_node[ii] else 0 for ii in range(len(visited_thresholds_per_node))])
    
    bottleneck_indices = np.where(max_thresholds_per_node == np.min(max_thresholds_per_node))[0]
    
    if len(bottleneck_indices) > 0:  # Check if any indices were found
        bottleneck_node = bottleneck_indices[0]
        thrs = []
        if len(visited_thresholds_per_node[bottleneck_node]) >= 2:
            thrs = np.linspace(visited_thresholds_per_node[bottleneck_node][-2], visited_thresholds_per_node[bottleneck_node][-1], 100, endpoint=True)
        elif len(visited_thresholds_per_node[bottleneck_node]) == 1:
            thrs = np.linspace(visited_thresholds_per_node[bottleneck_node][0], visited_thresholds_per_node[bottleneck_node][-1], 100, endpoint=True)
        
        visited_thresholds_of_bottleneck_node = []
        if thrs:
            visited_thresholds_of_bottleneck_node.append(thrs[0])
        
        for final_thr in thrs:
            list_of_infected_nodes_per_iter = run_cascade_single_population(adj_matrix, final_thr, bottleneck_node)
            if len(list_of_infected_nodes_per_iter[-1]) == adj_matrix.shape[0]:                
                visited_thresholds_of_bottleneck_node.append(final_thr)
            else:
                break
                
        return visited_thresholds_of_bottleneck_node[-1] if visited_thresholds_of_bottleneck_node else starting_thr
    else:
        return starting_thr

def main():
    adj_matrix = pd.read_csv('/home/gabridele/Desktop/connectome_sub-100206.csv', header=None).to_numpy().astype(float)
    zero_rows = np.where(np.sum(adj_matrix, 0) == 0)[0].tolist()
    adj_matrix_clean = np.delete(adj_matrix, zero_rows, axis=0)
    adj_matrix_clean = np.delete(adj_matrix_clean, zero_rows, axis=1)
    
    starting_thr = 0.0015
    thr = find_thr(adj_matrix_clean, starting_thr)
    
    seed_node_index_1 = np.random.randint(0, adj_matrix_clean.shape[0])
    seed_node_index_2 = np.random.randint(0, adj_matrix_clean.shape[0])
    
    _, activation_strengths_1 = run_cascade_single_population(adj_matrix_clean, thr, seed_node_index_1)
    _, activation_strengths_2 = run_cascade_single_population(adj_matrix_clean, thr, seed_node_index_2)
    
    result_matrix = np.zeros((adj_matrix_clean.shape[0], 2))  # 2 columns for A and B
    result_matrix[:, 0] = activation_strengths_1[-1]
    result_matrix[:, 1] = activation_strengths_2[-1]
    
    sum_A = np.sum(result_matrix[:, 0])
    sum_B = np.sum(result_matrix[:, 1])
    print('strength of seed A:', sum_A)
    print('strength of seed B:', sum_B)

    if sum_A > sum_B:
        print(f"Seed node A, aka {seed_node_index_1}, wins the competition with total strength {sum_A}.")
    elif sum_A < sum_B:
        print(f"Seed node B, aka {seed_node_index_2}, wins the competition with total strength {sum_B}.")
    else:
        print("It's a tie!")
    
if __name__ == "__main__":
    main()

strength of seed A: 2177.0
strength of seed B: 2302.0
Seed node B, aka 624, wins the competition with total strength 2302.0.
